# 1. (SKlearn으로) RF 트레이닝

In [2]:
import numpy as np
import pandas as pd
import random
import pickle
from time import time
from multiprocessing import Pool
import matplotlib.pyplot as plt 

import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier


import HEAAN as he

Custom tree의 training 고민할 필요 없이, wine도 SKLearn으로 train하고 가져오는걸로. 
1. load_from_sklearn_tree 작성.

In [3]:
sc = StandardScaler()

In [2]:
problem = ["iris", "wine", "tumor"][2]

if problem == "wine":
    df = pd.read_csv("./RF/winequality-red.csv")
    df["label"] = df.quality
    df = df.drop("quality", axis=1)
    label_map = {'good':0, 'bad':1}
    cleanup_names(df)
    wine_quality = df.label.value_counts(normalize=True)
    wine_quality = wine_quality.sort_index()

    def transform_label(value):
        if value <= 5:
            return "bad"
        else:
            return "good"

    df["label"] = df.label.apply(transform_label)
    # Standardize
    df.iloc[:,0:-1] = df.iloc[:,0:-1].apply(lambda x: 1. * (x-x.mean())/ x.std(), axis=0)

if problem == "tumor":
    data_dir = "/home/hoseung/Dropbox/DeepInsight/1. Random Forest Tumor Classification/data/"
    tumor_preprocessing(data_dir)
    X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data, data_label, test_size=0.33, stratify=data_label)
else:
    train_df, test_df = train_test_split(df, test_size=0.20)
    

In [10]:
sc = StandardScaler()
df = pd.read_csv("./RF/winequality-red.csv")
df["label"] = df.quality
df = df.drop("quality", axis=1)
label_map = {'good':0, 'bad':1}
#cleanup_names(df)
wine_quality = df.label.value_counts(normalize=True)
wine_quality = wine_quality.sort_index()

def transform_label(value):
    if value <= 5:
        return "bad"
    else:
        return "good"

df["label"] = df.label.apply(transform_label)

data = df.copy()
data_label = np.array(data['label'].tolist())
del data['label']
data = sc.fit_transform(data)

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data, data_label, test_size=0.33, stratify=data_label)

random_forest = RandomForestClassifier(n_estimators=3, max_depth = 3)

random_forest.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score
pred = random_forest.predict(X_test)
print("정확도 :{0:.3f}".format(accuracy_score(Y_test, pred)))

정확도 :0.746


In [7]:
from RF.utils_pure import *

In [11]:
RF = [extract_sk_dt(dt) for dt in random_forest]

dt = RF[0]

In [4]:
def fun(args):
    """
    Only non-leaf nodes should be passed/.
    각 노드에서 왼쪽으로 분기할 확률을 계산. 
    """
    threshold, ex_val = args
    n_element = len(ex_val)
    # Always generate an array of length Nslots.
    # And make sure its zero-initialized.
    v1 = np.zeros(nslots)
    v2 = np.zeros(nslots)

    v1[:n_element]=float(threshold)
    v2[:n_element]=np.copy(ex_val)
    assert v1.shape == v2.shape

    return 1/(1+np.exp(-(v1 - v2)))        


def gen_weight_forest(forest, example, ncpu=16):
    """
    Calculate weights for all questions for all examples for all trees.
    
    
    parameters
    ----------
    forest : group of trained trees
    test_df : test dataset in dataframe
    ncput : number of cores for multiprocessing
    """
    rf_branching_probs = []
    print(len(example))
    for dt in forest:
        #qlist=[]
        good_nodes = np.where(dt['feature'] >= 0)[0]# leaf nodes have 'features' == -2
        pool = Pool(min([len(good_nodes), ncpu]))
        #print(dt['threshold'][inode], "*******")
        branching_probs = pool.map(fun,
                                   [(dt['threshold'][inode],
                                     example[:,dt['feature'][inode]]) for inode in good_nodes])

        rf_branching_probs.append(np.stack(branching_probs))
    
    return rf_branching_probs

In [19]:
i=0
nslots = 4
rf_branching_probs = gen_weight_forest(RF, X_test[i*nslots:(i+1)*nslots], ncpu=1)

4


Ciphertext SIMD 하면 하나에 N example씩 계산. 

In [119]:
def predict_tree(tree, weights):

    ww = 1
    answers =[]
    def recurse(node, ww):
        if tree['children_left'][node] >=0 or tree['children_right'][node] >= 0:
            weight = weights[tree['branch']==node,:][0] # prb of this node for N example
            #print(weight)
            
            w1 = ww * weight
            w2 = ww * (1-weight)
            
            recurse(tree['children_left'][node], w1)
            recurse(tree['children_right'][node], w2)
        else:
            # value == Number of examples that fell in each class during the training phase.
            proba = tree['value'][node][0]
            proba /= np.sum(proba)
            #print(proba, proba[0])
            #print(ww)
            answers.append(ww)#
            #[prb * ww for prb in proba]) # 1D * scalar인 상태 
        
    recurse(0, ww)
    return answers

In [151]:
i_ex_l = 0
i_ex_r = 4

n_ex = i_ex_r - i_ex_l
n_class = 2

ans_rf=np.zeros((n_ex, n_class))


for itree, dt in enumerate(RF):
    

    prb_each_leaf = np.stack(predict_tree(dt, rf_branching_probs[itree][:,i_ex_l:i_ex_r]))

    i_most_probable_leaf = np.argmax(prb_each_leaf, axis=0)
    #print(i_most_probable_leaf)
    
    answer_from_leaf = dt['value'][dt['leaf'][i_most_probable_leaf]].squeeze()
    ans_rf += answer_from_leaf

In [158]:
label_map

{'good': 0, 'bad': 1}

In [168]:
dt['classes'] = ['good', 'bad']

i_class_exs = ans_rf.argmax(axis=1)
print(i_class_exs)

RF_answer = [dt['classes'][ii] for ii in i_class_exs]

[0 1 0 1]


In [169]:
RF_answer

['good', 'bad', 'good', 'bad']

# 여기서부터 다시 시작
이걸 이제... FHE로... 흐음..  
일단 모든 계산을 vectorized로 하긴 했는데.  
np.stack을 뭘로 바꿔야할지, 아님 SIMD로 자동으로 처리될지 의문   

In [1]:
def predict_tree_fhe(tree, weights):

    ww = 1
    answers =[]
    def recurse(node, ww):
        if tree['children_left'][node] >=0 or tree['children_right'][node] >= 0:
            weight = weights[tree['branch']==node,:][0] # prb of this node for N example
            #print(weight)
            
            w1 = ww * weight
            w2 = ww * (1-weight)
            
            recurse(tree['children_left'][node], w1)
            recurse(tree['children_right'][node], w2)
        else:
            # value == Number of examples that fell in each class during the training phase.
            proba = tree['value'][node][0]
            proba /= np.sum(proba)
            #print(proba, proba[0])
            #print(ww)
            answers.append(ww)#
            #[prb * ww for prb in proba]) # 1D * scalar인 상태 
        
    recurse(0, ww)
    return answers

In [1]:
import importlib
import numpy as np
from core.parameter import Parameters
import HEAAN as he

In [2]:
logp = 30 #
logq = logp+120 # Number of quantized bits.  Larger logq consumes more memory
logn = 2 # number of slots... 2^10 = 1024

n = 1*2**logn
nslots = n

#parms = {'logn':logn, 'n':n, 'logp':logp, 'logq':logq} # for ciphertext constructor

parms = Parameters(logn, 30, 40, logT=2)

sch = core.scheme.Scheme("he", parms)

sch.parms.logQ = 800

In [5]:
def fun_fhe(args):
    """
    Only non-leaf nodes should be passed/.
    각 노드에서 왼쪽으로 분기할 확률을 계산. 
    
    best if n_element == nslots.
    """
    threshold, x_enc = args
    # Always generate an array of length Nslots.
    # And make sure its zero-initialized.
    thres = np.zeros(nslots)
    thres[:nslots]=float(threshold)
    c_thres = sch.encrypt(thres)

    sch.scheme.bootstrapAndEqual(c_thres, sch.parms.logq, sch.parms.logQ, sch.parms.logT, sch.parms.logI)

    print(c_thres)
    print(x_enc)
    
    sch.scheme.subAndEqual(c_thres, x_enc)

    csig = he.Ciphertext()
    sch.algo.function(csig, c_thres, "Sigmoid", logp, 4)
        
    del c_thres, x_enc
    return csig


def fun(args):
    """
    
    global parameter
    ----------------
    nslots = Number of ciphertext slots, 2**logn
    """
    question, example = args
    feature_name, _, value = question.split(" ")

    n_element = len(example)
    # Always generate an array of length Nslots.
    # And make sure its zero-initialized.
    v1 = np.zeros(nslots)
    v2 = np.zeros(nslots)
    
    v1[:n_element]=float(value)
    v2[:n_element]=np.copy(example[feature_name])
    assert v1.shape == v2.shape
    
    c_crit = he.Ciphertext()
    sch.scheme.encrypt(c_crit, he.Double(v1), n, logp, logq)
    c_val = he.Ciphertext()
    sch.scheme.encrypt(c_val, he.Double(v2), n, logp, logq)

    sch.scheme.subAndEqual(c_crit, c_val)
    # No need to bcootstrap before Sigmoid. We have enough logq left
    csig = he.Ciphertext()
    algo.function(csig, c_crit, "Sigmoid", logp, 4)
    del c_crit, c_val
    return csig


In [21]:
def enc_and_boot(val):
    val = np.zeros(nslots)
    n_elements = nslots
    val[:n_elements] = example[:,dt['feature'][inode]]
    x_enc = sch.encrypt(val)
    sch.scheme.bootstrapAndEqual(x_enc, sch.parms.logq, sch.parms.logQ, sch.parms.logT, sch.parms.logI)
    return x_enc

# one node per ctxt 
i=0
example = X_test[i*nslots:(i+1)*nslots,:]

###########################################
rf_branching_probs = []
print(len(example))
#for dt in forest:
dt = RF[0]
##
#qlist=[]
#good_nodes = np.where(dt['feature'] >= 0)[0]# leaf nodes have 'features' == -2
good_nodes = dt['branch']
#inode = 0
ncpu = 8
pool = Pool(min([len(good_nodes), ncpu]))
#print(dt['threshold'][inode], "*******")

#####################################

#thres = dt['threshold'][inode]
#branching_probs = fun_fhe((thres, x_enc))

#####################################
branching_probs = pool.map(fun_fhe,
                           [(dt['threshold'][inode],
                             enc_and_boot(example[:,dt['feature'][inode]])) for inode in good_nodes])

#rf_branching_probs.append(np.stack(branching_probs))


#rf_branching_probs = gen_weight_forest(RF, x_enc, ncpu=1)

4


In [22]:
branching_probs

[<class.Ciphertext logp: 30 logq: 160 n: 4>,
 <class.Ciphertext logp: 30 logq: 160 n: 4>,
 <class.Ciphertext logp: 30 logq: 160 n: 4>,
 <class.Ciphertext logp: 30 logq: 160 n: 4>,
 <class.Ciphertext logp: 30 logq: 160 n: 4>,
 <class.Ciphertext logp: 30 logq: 160 n: 4>,
 <class.Ciphertext logp: 30 logq: 160 n: 4>]

In [1]:
def sig(x):
    return 1/(1+np.exp(-x))

정리

In [4]:
val = np.array([-0.57920651,  0.06993741, -0.63821955, -0.63821956])
threshold = 0.7

x_enc = sch.encrypt(val)
sch.scheme.bootstrapAndEqual(x_enc, sch.parms.logq, sch.parms.logQ, sch.parms.logT, sch.parms.logI)

thres = np.zeros(nslots)
thres[:nslots]=float(threshold)
c_thres = sch.encrypt(thres)
sch.scheme.bootstrapAndEqual(c_thres, sch.parms.logq, sch.parms.logQ, sch.parms.logT, sch.parms.logI)

sch.scheme.subAndEqual(c_thres, x_enc)

csig = he.Ciphertext()
sch.algo.function(csig, c_thres, "Sigmoid", logp, 4)
sch.decrypt(csig).real

array([0.77619316, 0.65228536, 0.78463355, 0.78462821])

In [26]:
csig = fun_fhe((thres, x_enc))

In [27]:
csig

<class.Ciphertext logp: 30 logq: 60 n: 4>

In [190]:
X_test.shape

(528, 11)

In [187]:
val.shape

(4,)

In [145]:
i_most_probable_leaf

array([4, 6, 4, 2])

In [126]:
prb_each_leaf

array([[0.46641031, 0.24655373, 0.46718433, 0.19734758],
       [0.04840474, 0.03685604, 0.06983696, 0.01706526],
       [0.13896301, 0.28904562, 0.17464828, 0.45242545],
       [0.07208145, 0.00798267, 0.02577502, 0.07060631],
       [0.02782622, 0.10038687, 0.06535441, 0.01774331],
       [0.02655041, 0.24488049, 0.05677091, 0.10074098],
       [0.1203593 , 0.04605017, 0.0679773 , 0.06139677],
       [0.09940456, 0.02824442, 0.07245278, 0.08267434]])

In [ ]:
answer[:]

In [117]:
np.stack(answer)

array([[[0.31369189, 0.16582375, 0.31421247, 0.13272935],
        [0.15271842, 0.08072998, 0.15297186, 0.06461824]],

       [[0.0468177 , 0.03564764, 0.06754723, 0.01650574],
        [0.00158704, 0.00120839, 0.00228974, 0.00055952]],

       [[0.03326528, 0.06919239, 0.0418077 , 0.10830262],
        [0.10569773, 0.21985323, 0.13284058, 0.34412283]],

       [[0.06449393, 0.00714239, 0.02306186, 0.06317407],
        [0.00758752, 0.00084028, 0.00271316, 0.00743224]],

       [[0.0113044 , 0.04078217, 0.02655023, 0.00720822],
        [0.01652182, 0.0596047 , 0.03880418, 0.01053509]],

       [[0.00235422, 0.02171354, 0.00503387, 0.0089327 ],
        [0.02419619, 0.22316695, 0.05173704, 0.09180828]],

       [[0.01851682, 0.00708464, 0.01045805, 0.00944566],
        [0.10184249, 0.03896552, 0.05751925, 0.05195111]],

       [[0.07144703, 0.02030067, 0.05207543, 0.05942219],
        [0.02795753, 0.00794374, 0.02037734, 0.02325216]]])

In [112]:
i_leaf = 0
nclass=2
n_example = 4

for i in range(nclass):
    leaf_prb_this_example=np.stack([ans_leaf[i] for ans_leaf in answer])
    for i_ex in range(n_example):
        prb_class_from_this_leaf = np.sum(tree_tot_prb_a_class[:,i_ex])
        print(prb_class_from_this_leaf)

0.5618912617999676
0.36768719039983516
0.5407468405494037
0.4057205323222125
0.43810873820003227
0.6323128096001649
0.45925315945059636
0.5942794676777874


In [113]:
tree_tot_prb_a_class[:,j]

array([0.06461824, 0.00055952, 0.34412283, 0.00743224, 0.01053509,
       0.09180828, 0.05195111, 0.02325216])

In [98]:
answer[7]

[array([0.07144703, 0.02030067, 0.05207543, 0.05942219]),
 array([0.02795753, 0.00794374, 0.02037734, 0.02325216])]

In [110]:
tmp = dt['value'][leaf_nodes][:,0,:]
all_prbs = [t / np.sum(t) for t in tmp]

In [111]:
all_prbs

[array([0.79487179, 0.20512821]),
 array([0.24912281, 0.75087719]),
 array([0.69798658, 0.30201342]),
 array([0.2455516, 0.7544484]),
 array([0.74390244, 0.25609756]),
 array([0.91891892, 0.08108108]),
 array([0.25, 0.75]),
 array([0., 1.])]

In [108]:
all_prbs  np.sum(all_prbs, axis=1)

ValueError: operands could not be broadcast together with shapes (8,2) (8,) 

In [83]:
branch_nodes = np.where(dt['feature'] >= 0)[0]
leaf_nodes = np.where(dt['feature'] < 0)[0]

In [85]:
len(leaf_nodes) = 2**tree_depth

8

In [82]:
len(dt['value'])

15

In [78]:
bp

0.7632796943667577

In [35]:
dt

{'children_left': array([ 1,  2,  3, -1, -1,  6, -1, -1,  9, 10, -1, -1, 13, -1, -1],
       dtype=int64),
 'children_right': array([ 8,  5,  4, -1, -1,  7, -1, -1, 12, 11, -1, -1, 14, -1, -1],
       dtype=int64),
 'feature': array([ 6,  7, 10, -2, -2,  9, -2, -2, 10,  3, -2, -2,  0, -2, -2],
       dtype=int64),
 'threshold': array([ 1.06529325, -0.39840026, -0.5378435 , -2.        , -2.        ,
        -0.07759537, -2.        , -2.        ,  0.77629802, -0.13395334,
        -2.        , -2.        , -1.50506139, -2.        , -2.        ]),
 'value': array([[[509., 562.]],
 
        [[379., 524.]],
 
        [[102., 222.]],
 
        [[ 31.,   8.]],
 
        [[ 71., 214.]],
 
        [[277., 302.]],
 
        [[208.,  90.]],
 
        [[ 69., 212.]],
 
        [[130.,  38.]],
 
        [[129.,  27.]],
 
        [[ 61.,  21.]],
 
        [[ 68.,   6.]],
 
        [[  1.,  11.]],
 
        [[  1.,   3.]],
 
        [[  0.,   8.]]]),
 'classes': array([0., 1.])}

## Load tumor data

In [10]:
RF = pickle.load(open("./RF/trained_RF.pickle", "rb"))
X_test, Y_test = pickle.load(open("./RF/tumor_testset.pickle", "rb"))

In [12]:
acc = []
for i in range(1000):
    rf_answer = RF_predict(X_test[i], RF)
    acc.append(rf_answer == int(Y_test[i]))
    
print("Accuracy", np.mean(acc))

Accuracy 0.611


In [14]:
np.power(2, 10)

1024

In [16]:
import seal

In [17]:
se = seal.EncryptionParameters(seal.scheme_type.ckks)

In [19]:
poly_modulus_degree = 16384
se.set_poly_modulus_degree(poly_modulus_degree)
se.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree,
                                              [37, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 37]))


In [22]:
se.scheme()

<scheme_type.ckks: 2>